In [43]:
!pip install selenium webdriver-manager

In [44]:
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from webdriver_manager.chrome import ChromeDriverManager

In [45]:
# ============================
# Importante para Appian + SSL
# ============================
def iniciar_driver():
    options = Options()
    options.add_argument("--start-maximized")
    options.add_argument("--ignore-certificate-errors")
    options.accept_insecure_certs = True
    
    # Opcional: menos ruido de logs
    options.add_argument("--log-level=3")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver


In [46]:
class BasePage:

    def __init__(self, driver):
        self.driver = driver
        self.wait = WebDriverWait(driver, 20)

    def click(self, locator):
        el = self.wait.until(EC.element_to_be_clickable(locator))
        self.scroll_into_view(el)
        self.safe_click(el)

    def write(self, locator, text):
        element = self.wait.until(EC.visibility_of_element_located(locator))
        self.scroll_into_view(element)
        element.clear()
        element.send_keys(text)

    def get_text(self, locator):
        return self.wait.until(EC.visibility_of_element_located(locator)).text

    # === NUEVOS MÉTODOS ===
    def scroll_into_view(self, element):
        try:
            self.driver.execute_script(
                "arguments[0].scrollIntoView({block:'center', inline:'nearest'});", element
            )
            # opcional: mover mouse encima ayuda a evitar overlays pequeños
            from selenium.webdriver.common.action_chains import ActionChains
            ActionChains(self.driver).move_to_element(element).perform()
        except Exception:
            pass

    def safe_click(self, element):
        try:
            element.click()
        except Exception:
            # fallback por si hay overlay/hitbox extraño
            self.driver.execute_script("arguments[0].click();", element)


In [47]:
# ============================
# Página de Información Básica
# ============================
class BasicInfoPage(BasePage):

    # Campos (ajustar si el DOM cambia)
    DOCUMENT = (By.XPATH, "//input[contains(@placeholder,'Ingresa tu número de documento') or contains(@id,'6aca9fae4a0f9f94b237257027129026')]")
    ISSUE_DATE = (By.XPATH, "//input[contains(@placeholder,'mm/dd/yyyy') or contains(@id,'7738f10e86f4493d96d8ff36e455e3d8')]")
    FIRSTNAME = (By.XPATH, "//input[contains(@placeholder,'Ingresa tu nombre') or contains(@id,'76a3fe31d079db8956e62e820ee17153')]")
    MIDDLENAME = (By.XPATH, "//input[contains(@placeholder,'Ingresa tu segundo nombre') or contains(@id,'3e4cdfc141e0f9cfe3f88fc12a698935')]")
    LASTNAME = (By.XPATH, "//input[contains(@placeholder,'Ingresa tu apellido paterno') or contains(@id,'b6be9a84640da70a422c21bd64400d8e')]")
    SECOND_LASTNAME = (By.XPATH, "//input[contains(@placeholder,'Ingresa tu apellido materno') or contains(@id,'c3baa7c468dd593eb5c885c59c6e6c18')]")
    EMAIL = (By.XPATH, "//input[contains(@placeholder,'tuemail@dominio.com') or contains(@id,'eaf278c0a6a093598c09eabbd1dd8269')]")
    PHONE = (By.XPATH, "//input[contains(@placeholder,'Ingresa tu número de celular') or contains(@id,'4276533d0182c699afbe6d30b382d619')]")

    #SELECTORES CSS PARA LOS CHECKBOXES ===
    CHECK_DATA = (By.CSS_SELECTOR,".FieldLayout---field_layout:nth-child(3) .CheckboxGroup---choice_label:nth-child(2)")
    CHECK_TERMS = (By.CSS_SELECTOR,".FieldLayout---field_layout:nth-child(4) .CheckboxGroup---choice_label:nth-child(2)")  
    
    CONTINUE_BTN = (By.XPATH, "//button[contains(.,'Continuar')]")

    # ------------------------------------
    # Llenar información básica
    # ------------------------------------
    def fill_basic_information(self):
        self.write(self.DOCUMENT, "1032444894")
        self.write(self.ISSUE_DATE, "08/04/2009")
        self.write(self.FIRSTNAME, "Juan")
        self.write(self.MIDDLENAME, "Manuel")
        self.write(self.LASTNAME, "Davila")
        self.write(self.SECOND_LASTNAME, "Davila")
        self.write(self.EMAIL, "jdavilaargel@gmail.com")
        self.write(self.PHONE, "3183687029")

    # ------------------------------------
    # Aceptar políticas (versión V3)
    # ------------------------------------
    def accept_policies(self):
        # Clic directo por CSS (mucho más confiable que XPath)
        self.click(self.CHECK_DATA)
        time.sleep(0.2)  # evitar doble renderizado
        self.click(self.CHECK_TERMS)
        time.sleep(0.2)
        
    # ------------------------------------
    # Continuar (dispara reCAPTCHA invisible)
    # ------------------------------------
    def continue_process(self):
        current_url = self.driver.current_url
        # Si hay un overlay/spinner, puedes esperar a que desaparezca
        # self.wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, ".spinner, .loading, .a-Spinner")))
        self.click(self.CONTINUE_BTN)
        self.wait.until(EC.url_changes(current_url))

    

In [48]:
# ============================
# Flujo completo
# ============================
driver = iniciar_driver()
driver.get("https://bancognbsudamerissa-test.appianportals.com/16dbbd19-0d2a-4bad-8d03-cabf21882c8b-apertura-producto?numeroSolicitud=ONB305202602101054")

basic = BasicInfoPage(driver)
basic.fill_basic_information()
basic.accept_policies()     
basic.continue_process()

print("✔ Pantalla completada correctamente (captcha validado por backend)")

✔ Pantalla completada correctamente (captcha validado por backend)
